
$\qquad$ $\qquad$$\qquad$  **TDA231/DIT370 Discrete Optimization: Home Assignment 2 -- Integer LP and Relaxation** <br />
$\qquad$ $\qquad$$\qquad$                   **Grader: Elisabet Lobo** <br />
$\qquad$ $\qquad$$\qquad$                   **Due Date: 8th Feb** <br />
$\qquad$ $\qquad$$\qquad$                   **Submitted by: Name, Personal No., Email** <br />


---


General guidelines:
*   All solutions to theoretical and pratical problems must be submitted in this ipynb notebook, and equations wherever required, should be formatted using LaTeX math-mode.
*   All discussion regarding practical problems, along with solutions and plots should be specified in this notebook. All plots/results should be visible such that the notebook do not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.
*   Your name, personal number and email address should be specified above.
*   All tables and other additional information should be included in this notebook.
*   Before submitting, make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.


# Question 1.
(5 points) In the previous assignment, we solved the transportation problem of the space colonies using LP. Explain why the solution was integral with a proof.

# Question 2.

Recall the Minimum Weight Vertex Cover (VC) Problem: Given an undirected graph $G=(V, E)$, with node set $V$ and edge set $E$, where each node has a positive weight $w(v)$ associated with it (see figure), the goal is to select a subset $V'\subseteq V$ of nodes such that every edge has at least one node incident to it, and the total selected node weight $\sum_{v\in V'} w(v)$ is minimized. 

* (4 points) Formulate the ILP for the VC problem for the example below, and solve it using **CVXPY** integer solver, for instance, `myVar = cp.Variable(<dim>, integer=True)`.
* (2 points) Pass to the LP relaxation and solve it using **CVXPY** and comment on the relation between the two solutions.
* (2 points) Apply the rounding rule discussed in class to the optimal LP solution to obtain a solution to the ILP and compare it to the optimal ILP solution.

![alt text](https://drive.google.com/uc?id=1NDgRt1SqvTy_hSnAQmGoSE8peMNFmI6C)


## Question 2.1

**Variables:** <br />
$x_v$ for every node v, such that $v \in V$ and $x_v \in \{0, 1\}$, where $x_v = 1$ if $v \in V'$ and $x_v = 0$ if $v \not \in V'$ $.

**Objective function:** <br />
min $\sum_{v \in V} w(v)*x_v$

**s.t** <br />
$x_u + x_v \geq 1 \quad \forall (u,v) \in E$ <br />
$x_v \in \{0, 1\} \quad \forall v \in V$

The ILP is solved using cvxpy as following.

In [3]:
import cvxpy as cp
import numpy as np

# nodes
x = cp.Variable(6, integer=True)

# weights 
w = np.array([4,2,3,2,1,4]) 

# incidence matrix
A = np.array([[1, 1, 0, 0, 0, 0],  # 4-2 (top)
              [1, 0, 1, 0, 0, 0],  # 4-2 (centre)
              [1, 0, 0, 1, 0, 0],  # 4-3
              [0, 1, 0, 1, 0, 0],  # 2-2
              [0, 0, 1, 1, 0, 0],  # 3-2
              [0, 0, 1, 0, 1, 0],  # 3-1
              [0, 0, 0, 1, 0, 1]]) # 2-4 (bottom)

# defining problem
objective = cp.Minimize(w.T@x)
constraints = [A@x >= np.ones(7), x <= 1, x >= 0]
prob = cp.Problem(objective, constraints)

# print result
result = prob.solve()
print("x values: ", x.value)
print("optimal solution: ", result)

x values:  [0. 1. 1. 1. 0. 0.]
optimal solution:  7.0


## Question 2.2
LP relaxation of 2.1 is performed as following.

We can observe that the optimal solution of the LP-relaxation is very close (~$10^{-8}$) to the ILP solution. However, in ILP solution, three distinct nodes are selected whereas in the LP-relaxation, fractions of all nodes are selected. Thus, the ILP solution is an invalid vertex cover. 

In [14]:
import cvxpy as cp
import numpy as np

# nodes without integer constraint
x1 = cp.Variable(6, nonneg=True)

# defining problem
objective1 = cp.Minimize(w.T@x1)
constraints1 = [A@x1 >= np.ones(7), x1 <= 1, x1 >= 0]
prob1 = cp.Problem(objective1, constraints1)

# print result
result1 = prob1.solve()
print("x values: ", x1.value)
print("optimal solution: ", result1)

x values:  [4.99582079e-01 5.00417921e-01 5.00417921e-01 1.00000000e+00
 4.99582079e-01 2.65500505e-10]
optimal solution:  6.999999998934636


## Question 2.3
We have rounded the values of every $x_i$ to the nearest integer, i.e. $x_i \geq 0.5$ are rounded to 1 and $x_i < 0.5$ is rounded to 0. The rounded x-vector is then [0. 1. 1. 1. 0. 0.] and we observe that rounded LP-relaxation correspond to the selected nodes in the ILP solution. Using the rounded x-vector, the optimal value of the rounded solution is computed to 7, which is equal to the optimal value of the ILP solution.



# Question 3.

Consider a number of interpreters (Olof, Petra, Qamar,
  Rachel, Soren and Tao), as well as a set of languages (Arab,
  Bengali, Cantonese, Dutch, English, French and German). Each
  interpreter speaks a number of different languages (abbreviated by
  first letter), and has a certain per-diem integer cost:

\begin{array}{lll}
Interpreter & Languages & Cost\\
O & ABD & 3\\
P & C & 1\\
Q & CDG & 1\\
R & B & 2\\
S & G & 4\\
T & EF & 1\\
\end{array}

* (2 points) A *hypergraph* is a structure $H = (V,E)$ where $V$ is a set of vertices and $E$ is a collection of subsets of $V$. The special case when all subsets $e \in E$ have size exactly $2$ corresponds to the familiar case of a graph. A vertex cover in such a hypergraph is a subset $U \subseteq V$ such that $e \cap U \not = \emptyset$ for each $e \in E$ (note that this reduces to the usual vertex cover in graphs). Show that the problem of finding interpreters can be formulated as a vertex cover problem in a sutable hypergraph.
* (4 points) Develop a ILP formulation to finding the vertex cover of minimum cost in a hypergraph. The hypergraph can be represented as a $|V| \times |E|$ binary matrix $A$ where $A[i,j] = 1$ iff vertex $i$ is in edge $j$ and 0 otherwise. The costs for vertices are in an array $\texttt{c}$ where the cost of picking vertex $i$ is $c[i]$. Use the ILP formulation for the VC problem to hire the cheapest set of interpreters such that all languages are covered. Input the data above manuallly and solve it using **CVXPY**'s integer solver.
* (2 points) Pass to the LP relaxation and solve it using **CVXPY**.
* (2 points) Explain why the two solutions above are same (different).


## Question 3.1



## Question 3.2

**Objective function:** <br/>
$\min \sum_{e \in E} x_e * w_e$

**Constraints:** <br />
$\sum_{e : v \in e} x_e \geq 1  \quad \forall v \in V$ <br />
$x_e \in \{0, 1\} \quad \forall e \in E$


We calculate our ILP solution below. The cheapest choice of interpretors is then O,Q and T. 

In [6]:
# Subsets/Intepretors
x = cp.Variable(6, boolean=True)

# costs 
w = np.array([3,1,1,2,4,1]) 

#Incidence matrix
A = np.array([
    #A,B,C,D,E,F,G
    [1,1,0,1,0,0,0], #O
    [0,0,1,0,0,0,0], #P
    
    [0,0,1,1,0,0,1], #Q
    [0,1,0,0,0,0,0], #R
    [0,0,0,0,0,0,1], #S
    [0,0,0,0,1,1,0]  #T
])

# defining problem
objective = cp.Minimize(w.T@x)
constraints = [A.T@x >= np.ones(7), x <= 1, x >= 0]
prob = cp.Problem(objective, constraints)

# print result
result = prob.solve()
print("x values: ", x.value)
print("optimal solution: ", result)


x values:  [1. 0. 1. 0. 0. 1.]
optimal solution:  5.0


## Question 3.3

In [7]:
# subsets without integer constraint
x1 = cp.Variable(6, nonneg=True)

# defining problem
objective1 = cp.Minimize(w.T@x1)
constraints1 = [A.T@x1 >= np.ones(7), x1 <= 1, x1 >= 0]
prob1 = cp.Problem(objective1, constraints1)

# print result
result1 = prob1.solve()
print("x values: ", x1.value)
print("optimal solution: ", result1)

x values:  [1.00000000e+00 1.07010319e-09 1.00000000e+00 3.00389586e-10
 0.00000000e+00 1.00000000e+00]
optimal solution:  5.000000000472339


## Question 3.4

We can see that our matrix $A$ is totally unimodular since:
* All entries are eiter 0 or 1
* any column has at most 2 non-zeros (ones)
* rows can be partitioned into two subsets Rows1= {O,P,S}, Rows2={Q,R,T} such that no non-zeros with same sign are in the same subset.

From Hoffman-Kruskal Theorem, we can then conclude that the entries in x will be integers. In practice, this means that the optimal solution from the LP relaxation will be identical to the optimal ILP solution.

Observe that the problem is **not** totally unimodular in general. In this specific instance of the problem, each language is represented in at most 2 interpreters, which corresponds to the second criterion of totally unimodularity.

# Question 4. 
Consider the ILP and its LP relaxation corresponding to the VC problem for the graph $G$ given in the data file. This is a ***random graph*** $G(n,p)$ with $n=100$ vertices generated as follows: for each pair of vertices **independently**, we add an edge with probability $p=0.1$ (so the graph has about 1000 edges).

* **a**. (2 points) Find the optimal solution using **CVXPY**'s integer solver.
* **b**. (2 points) Solve the LP relaxation using **CVXPY** and apply the rounding rule discussed in class to obtain a vertex cover. Compare it to the optimal solution in part (a).
* **c**. (6 points) Consider the following rounding rule: we build up the vertex cover incrementally starting with $S:= \emptyset$. Now consider the edges in $G$ in any order. If an edge $(u,v)$ is already covered by a vertex in $S$, do nothing. Otherwise add to $S$ the vertex $u$ if $x^*(u) \geq x^*(v)$, or $v$ otherwise (where ${\bf x}^*$ is the LP optimum solution computed in part (b).  Comment why this also results in a vertex cover and has cost no more than that corresponding to the rounding rule in part (b). Compare the cost of the solution produced by this rule to the optimal solution.

In [54]:
# read file to numpy array 
A = np.loadtxt('graph.txt')
n = A.shape[0]

# Variables for each node
x = cp.Variable(n, integer=True)


#4.a ILP
objective = cp.Minimize(x.T@np.ones(n))

constraints = [A.T@x >= np.ones(7), x <= 1, x >= 0]
prob = cp.Problem(objective, constraints)

# print result
result = prob.solve()
print("x values: ", x.value)
print("optimal solution: ", result)
    

ValueError: Cannot broadcast dimensions  (7,) (100,)